In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [3]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JI,JE,a,b=3.5,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
# random gain and symmetricity
gaverage=1.0
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
etaset=1*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
signeta=np.zeros(3)
signeta[0::2]=1
signeta[1]=1
nrank,ntrial,neta=1,2,len(symetaseries)
''' C0 rules '''
# xax=np.arange(NE*2)
# XX,YY=np.meshgrid(xax,xax)
# idxdowntriangle=np.where(XX>YY)
# idxuptriangle=np.where(XX<YY)

Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
pearsoncorr_rho=np.zeros((neta,ntrial,2,nrank))
covnmEI=np.zeros((neta,ntrial,2,nrank))
eta_est,eta_num=np.zeros((neta,ntrial)),np.zeros((neta,ntrial,2))
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
for idxeta, eta in enumerate(symetaseries):
    etaset[1]=eta
    for ktrial in range(ntrial):
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        # ''' HOMOGENEOUS ETA '''
        # Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
        # # Xinit[idxuptriangle]=XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta) # positive degree of symmetry
        # Xinit[idxuptriangle]= -XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta) # negative degree of symmetry
        ''' HETEROGENEOUS ETA '''
        # EE#
        Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # EI/IE#
        Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # EE#
        Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0
        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        
        ''' '''
        longaxis,shortaxis=(1+eta)*radius_g ,(1-eta)*radius_g 
        xr = longaxis*np.cos(theta)
        yr = shortaxis*np.sin(theta)
        # overall
        J = X.copy()+Am.copy()
        eta_num[idxeta,ktrial,0]=np.mean(np.mean(J.copy()*J.copy().T))-np.mean(np.mean(J.copy()))*np.mean(np.mean(J.copy().T))
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symmetricityX,eta_num[idxeta,ktrial,0]*2*NE,symtheofull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()
        lambda_est=neig.copy().T@meig.copy()
        Jtilde=meig.copy()@neig.copy().T
        eigvestJ,eigrvecestJ=la.eig(Jtilde)
        print('eigvalue:',eigvJ[:nrank],'; est lambda:',lambda_est,'; eigenvalues est:',eigvestJ[:nrank])
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmE[irank]=np.mean(mEmat)
            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmI[irank]=np.mean(mImat)  
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannE[irank]=np.mean(nEmat)
            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        '''
        calculate overlap/noise covariance
        '''
        covnmEI[idxeta,ktrial,0,0]=np.sum(np.squeeze(noiseE[:,0,0])*np.squeeze(noiseE[:,1,0]))/NE
        covnmEI[idxeta,ktrial,1,0]=np.sum(np.squeeze(noiseI[:,0,0])*np.squeeze(noiseI[:,1,0]))/NE
        '''
        Calculate Pearson correlation and determine the sign of rotation
        '''
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
        ## pearson correlation 
        normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
        normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
        pearsoncorr_rho[idxeta,ktrial,0,0],pearsoncorr_rho[idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI
        
        '''
        calculate eta
        '''
        etamat=np.array([[etaset[0],etaset[1]],[etaset[1],etaset[2]]])
        diageta,_=la.eig(etamat)
        mean_eta=meanmE[0]*meannE[0]*covnmEI[idxeta,ktrial,0,0]*diageta[0]/np.sum(diageta)+meanmI[0]*meannI[0]*covnmEI[idxeta,ktrial,1,0]*diageta[1]/np.sum(diageta)#+meanmE[0]*meannE[0]*covnmEI[idxeta,ktrial,1,0]+meanmI[0]*meannI[0]*covnmEI[idxeta,ktrial,0,0]
        mean_eta/=(4.0/(NE))
        std_eta=(covnmEI[idxeta,ktrial,0,0]+covnmEI[idxeta,ktrial,1,0])**2/(4.0/NE)
        eta_est[idxeta,ktrial]=mean_eta#+std_eta
        ''' numerical Jtilde '''
        eta_num[idxeta,ktrial,1]=np.mean(np.mean(Jtilde.copy()*Jtilde.copy().T))-np.mean(np.mean(Jtilde.copy()))*np.mean(np.mean(Jtilde.copy().T))
        # print('ETA:',eta_est[idxeta,ktrial]*(2*NE),'; num:',eta_num[idxeta,ktrial,1]*(2*NE)*NE)
        # print('meanpart:',mean_eta)
        # print('stdpart:',std_eta)

        ## ee 
        NE2=NE**2
        nEm,nEn=np.squeeze(noiseE[:,0,0]),np.squeeze(noiseE[:,1,0])
        nIm,nIn=np.squeeze(noiseI[:,0,0]),np.squeeze(noiseI[:,1,0])
        sigmnE,sigmnI=np.sum(nEm*nEn)/NE,np.sum(nIm*nIn)/NE
        mEm,mEn=meanmE[0],meannE[0]
        mIm,mIn=meanmI[0],meannI[0]
        JtildeT=Jtilde.copy().T
        #ee
        theoEE=(mEm*mEn)**2*NE2+sigmnE*(mEm*mEn)*2*NE2+sigmnE*sigmnE*NE2
        numEE=np.sum(np.sum(Jtilde[:NE,:NE].copy()*JtildeT[:NE,:NE].copy()))
        #ei
        theoEI=(mEm*mIn)*(mIm*mEn)*NE2+sigmnE*(mIm*mIn)*NE2+sigmnI*(mEm*mEn)*NE2+sigmnE*sigmnI*NE2
        numEI=np.sum(np.sum(Jtilde[:NE,NE:].copy()*JtildeT[:NE,NE:].copy()))
        #ei
        theoIE=(mIm*mEn)*(mEm*mIn)*NE2+sigmnE*(mIm*mIn)*NE2+sigmnI*(mEm*mEn)*NE2+sigmnE*sigmnI*NE2
        numIE=np.sum(np.sum(Jtilde[NE:,:NE].copy()*JtildeT[NE:,:NE].copy()))
        #II
        theoII=(mIm*mIn)**2*NE2+sigmnI*(mIm*mIn)*2*NE2+sigmnI*sigmnE*NE2
        numII=np.sum(np.sum(Jtilde[NE:,NE:].copy()*JtildeT[NE:,NE:].copy()))
        ## mean
        theomean=((mIm*mEn)*(mEm*mIn)*NE2+(mEm*mIn)*(mIm*mEn)*NE2+(mEm*mEn)**2*NE2+(mIm*mIn)**2*NE2)/4
        nummean = np.mean(np.mean(Jtilde.copy()))*NE*np.mean(np.mean(Jtilde.copy().T))*NE
        thoeta_est=(theoEE+theoEI+theoIE+theoII-theomean*4)/4/NE2#[idxeta,ktrial]
        numeta_est=(numEE+numEI+numII+numIE-nummean*4)/4/NE2#[idxeta,ktrial]
        print('mean sub:',thoeta_est*2*NE,numeta_est*2*NE,'; num:',eta_num[idxeta,ktrial,1]*2*NE)

normalization m & n
1.0000000000000004 5.147815070493501
eigenvalues: [-2.5+0.j]
gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
compare the first and the last, mean, terms: (6.249999999999936+0j) 6.2499999999999725
eta full: 0.5001159174362994 0.5000937776760718 0.5


KeyboardInterrupt: 

## the second rank in a recurrent neural network with rank 2 structural connectivity

In [26]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JI,JE,a,b=3.5,1.0,-3.5,1.0 # already recording -3.5# 2.0
JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
# random gain and symmetricity
gaverage=1.0
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
etaset=1*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
signeta=np.zeros(3)
signeta[0::2]=1
signeta[1]=1
nrank,ntrial,neta=1,2,len(symetaseries)
''' C0 rules '''
# xax=np.arange(NE*2)
# XX,YY=np.meshgrid(xax,xax)
# idxdowntriangle=np.where(XX>YY)
# idxuptriangle=np.where(XX<YY)

Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
pearsoncorr_rho=np.zeros((neta,ntrial,2,nrank))
covnmEI=np.zeros((neta,ntrial,2,nrank))
eta_est,eta_num=np.zeros((neta,ntrial)),np.zeros((neta,ntrial,2))
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
for idxeta, eta in enumerate(symetaseries):
    etaset[1]=eta
    for ktrial in range(ntrial):
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        # ''' HOMOGENEOUS ETA '''
        # Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
        # # Xinit[idxuptriangle]=XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta) # positive degree of symmetry
        # Xinit[idxuptriangle]= -XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta) # negative degree of symmetry
        ''' HETEROGENEOUS ETA '''
        # EE#
        Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # EI/IE#
        Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # EE#
        Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0+(a-b)**2/(NE*2)
        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        
        ''' '''
        longaxis,shortaxis=(1+eta)*radius_g ,(1-eta)*radius_g 
        xr = longaxis*np.cos(theta)
        yr = shortaxis*np.sin(theta)
        # overall
        J = X.copy()+Am.copy()
        eta_num[idxeta,ktrial,0]=np.mean(np.mean(J.copy()*J.copy().T))-np.mean(np.mean(J.copy()))*np.mean(np.mean(J.copy().T))
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symmetricityX,eta_num[idxeta,ktrial,0]*2*NE,symtheofull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()
        lambda_est=neig.copy().T@meig.copy()
        Jtilde=meig.copy()@neig.copy().T
        eigvestJ,eigrvecestJ=la.eig(Jtilde)
        print('eigvalue:',eigvJ[:nrank],'; est lambda:',lambda_est,'; eigenvalues est:',eigvestJ[:nrank])
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        '''
        calculate overlap/noise covariance
        '''
        covnmEI[idxeta,ktrial,0,0]=np.sum(np.squeeze(noiseE[:,0,0])*np.squeeze(noiseE[:,1,0]))/NE
        covnmEI[idxeta,ktrial,1,0]=np.sum(np.squeeze(noiseI[:,0,0])*np.squeeze(noiseI[:,1,0]))/NE
        '''
        Calculate Pearson correlation and determine the sign of rotation
        '''
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
        ## pearson correlation 
        normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
        normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
        pearsoncorr_rho[idxeta,ktrial,0,0],pearsoncorr_rho[idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI
        '''
        calculate eta
        '''
        etamat=np.array([[etaset[0],etaset[1]],[etaset[1],etaset[2]]])
        diageta,_=la.eig(etamat)
        mean_eta=meanmE[0]*meannE[0]*covnmEI[idxeta,ktrial,0,0]*diageta[0]/np.sum(diageta)+meanmI[0]*meannI[0]*covnmEI[idxeta,ktrial,1,0]*diageta[1]/np.sum(diageta)#+meanmE[0]*meannE[0]*covnmEI[idxeta,ktrial,1,0]+meanmI[0]*meannI[0]*covnmEI[idxeta,ktrial,0,0]
        mean_eta/=(4.0/(NE))
        std_eta=(covnmEI[idxeta,ktrial,0,0]+covnmEI[idxeta,ktrial,1,0])**2/(4.0/NE)
        eta_est[idxeta,ktrial]=mean_eta#+std_eta
        ''' numerical Jtilde '''
        eta_num[idxeta,ktrial,1]=np.mean(np.mean(Jtilde.copy()*Jtilde.copy().T))-np.mean(np.mean(Jtilde.copy()))*np.mean(np.mean(Jtilde.copy().T))
        # print('ETA:',eta_est[idxeta,ktrial]*(2*NE),'; num:',eta_num[idxeta,ktrial,1]*(2*NE)*NE)
        # print('meanpart:',mean_eta)
        # print('stdpart:',std_eta)

        ## ee 
        NE2=NE**2
        nEm,nEn=np.squeeze(noiseE[:,0,0]),np.squeeze(noiseE[:,1,0])
        nIm,nIn=np.squeeze(noiseI[:,0,0]),np.squeeze(noiseI[:,1,0])
        sigmnE,sigmnI=np.sum(nEm*nEn)/NE,np.sum(nIm*nIn)/NE
        mEm,mEn=meanmE[0],meannE[0]
        mIm,mIn=meanmI[0],meannI[0]
        JtildeT=Jtilde.copy().T
        #ee
        theoEE=(mEm*mEn)**2*NE2+sigmnE*(mEm*mEn)*2*NE2+sigmnE*sigmnE*NE2
        numEE=np.sum(np.sum(Jtilde[:NE,:NE].copy()*JtildeT[:NE,:NE].copy()))
        #ei
        theoEI=(mEm*mIn)*(mIm*mEn)*NE2+sigmnE*(mIm*mIn)*NE2+sigmnI*(mEm*mEn)*NE2+sigmnE*sigmnI*NE2
        numEI=np.sum(np.sum(Jtilde[:NE,NE:].copy()*JtildeT[:NE,NE:].copy()))
        #ei
        theoIE=(mIm*mEn)*(mEm*mIn)*NE2+sigmnE*(mIm*mIn)*NE2+sigmnI*(mEm*mEn)*NE2+sigmnE*sigmnI*NE2
        numIE=np.sum(np.sum(Jtilde[NE:,:NE].copy()*JtildeT[NE:,:NE].copy()))
        #II
        theoII=(mIm*mIn)**2*NE2+sigmnI*(mIm*mIn)*2*NE2+sigmnI*sigmnE*NE2
        numII=np.sum(np.sum(Jtilde[NE:,NE:].copy()*JtildeT[NE:,NE:].copy()))
        ## mean
        theomean=((mIm*mEn)*(mEm*mIn)*NE2+(mEm*mIn)*(mIm*mEn)*NE2+(mEm*mEn)**2*NE2+(mIm*mIn)**2*NE2)/4
        nummean = np.mean(np.mean(Jtilde.copy()))*NE*np.mean(np.mean(Jtilde.copy().T))*NE
        thoeta_est=(theoEE+theoEI+theoIE+theoII)/4/NE2#-theomean*4)/4/NE2#[idxeta,ktrial]
        numeta_est=(numEE+numEI+numII+numIE-nummean*4)/4/NE2#[idxeta,ktrial]
        # ratio_=symtheofull/(thoeta_est-theomean/NE2)
        ratio_=symtheofull/thoeta_est/2/NE
        print('ratio 2nd:',ratio_)
        print('mean sub:',thoeta_est*2*NE,numeta_est*2*NE,'; num:',eta_num[idxeta,ktrial,1]*2*NE)


# std_eta=(covnmEI[idxeta,ktrial,0,0]+covnmEI[idxeta,ktrial,1,0])**2/(4.0/NE)




normalization m & n
1.0000000000000004 5.147815070493483
eigenvalues: [-4.5+0.j]
gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
compare the first and the last, mean, terms: (0.010124999999999856+0j) 1.5762150861130516e-31
eta full: 0.5007494177169973 0.5107519108615795 0.510125
eigvalue: [-4.58711921+0.j] ; est lambda: [[-4.58711921-2.03942808e-14j]] ; eigenvalues est: [-4.58711921-2.03942808e-14j]
ratio 2nd: 48.483146964454704
mean sub: 0.010521697371954773 (0.01052070124191018+9.356197678707817e-17j) ; num: 0.01052070124191019
compare the first and the last, mean, terms: (0.010124999999999856+0j) 1.5762150861130516e-31
eta full: 0.5005702546589021 0.5108233763062244 0.510125
eigvalue: [-4.62942692+0.j] ; est lambda: [[-4.62942692-3.05812809e-13j]] ; eigenvalues est: [-4.62942692-3.05812809e-13j]
ratio 2nd: 47.603003168725344
mean sub: 0.010716235658323898 (0.010715761379334176+1.415738615572335e-15j) ; num: 0.010715761379334178
compare the first and the last, mean, te

In [20]:
print('a:',2*a/np.sqrt(2*NE),';real:',neigvecAm[0,0])
print('b:',2*b/np.sqrt(2*NE),';real:',neigvecAm[NE,0])
print('mumE:',1/np.sqrt(2*NE),';real:',meigvecAm[0,0])
print('mumI:',-1/np.sqrt(2*NE),';real:',meigvecAm[NE,0])

a: -0.15652475842498528 ;real: (-0.15652475842498534+0j)
b: 0.044721359549995794 ;real: (0.04472135954999591-0j)
mumE: 0.022360679774997897 ;real: (0.022360679774997668+0j)
mumI: -0.022360679774997897 ;real: (-0.022360679774997897+0j)


In [31]:
fig,ax=plt.subplots(figsize=(6,6))
# ax.plot(symetaseries,np.mean(np.squeeze(pearsoncorr_rho[:,:,0,0]),axis=1),linewidth=1.5,color='red',alpha=0.5)
# ax.plot(symetaseries,np.mean(np.squeeze(pearsoncorr_rho[:,:,1,0]),axis=1),linewidth=1.5,color='blue',alpha=0.5)
ax.plot(symetaseries,np.mean(np.squeeze(thetamaxseries[:,:,0,0]),axis=1),linewidth=1.5,color='red',alpha=0.5)
ax.plot(symetaseries,np.mean(np.squeeze(thetamaxseries[:,:,1,0]),axis=1),linewidth=1.5,color='blue',alpha=0.5)


## Perturbation Theory for Partially Symmetric Matrix

In [7]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
JI,JE,a,b=3.5,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
# random gain and symmetricity
gaverage=1.0
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
etaset=1*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
signeta=np.zeros(3)
signeta[0::2]=1
signeta[1]=1
nrank,ntrial,neta=1,2,len(symetaseries)
''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
XsymT=Xsym.copy().T
Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
Xinit = Xsym.copy()
''' HETEROGENEOUS ETA '''
# EE#
Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
# EI/IE#
Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
# EE#
Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
''' THEORETICAL FULL ETA '''
symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0+(a-b)**2/(NE*2)
# ''' homogeneous '''
X=Xinit.copy()
X*=(gaverage/np.sqrt(2))
# overall
J = X.copy()+Am.copy()
eta_num[idxeta,ktrial,0]=np.mean(np.mean(J.copy()*J.copy().T))-np.mean(np.mean(J.copy()))*np.mean(np.mean(J.copy().T))
'''
Adding extra value mean-value can not be neglected
'''
barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
# the first term
symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
symavg/=(2*NE)
avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
print('compare the first and the last, mean, terms:',symavg,avgmean)
print('eta full:',symmetricityX,eta_num[idxeta,ktrial,0]*2*NE,symtheofull)
# properties
eigvJ,eigrvecJ=la.eig(J)
inveigrvecJ=la.inv(eigrvecJ)
meig = np.squeeze(eigrvecJ[:,:].copy())
# meig = np.reshape(meig,(NE*2,nrank))
neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
neig=neig.copy().T
# neig = np.reshape(neig.T,(NE*2,nrank))
for i in range(nrank):
    neig[:,i]*=eigvJ[i]
for j in range(1,N):
    neig[:,j]*=eigvJ[j]

DiagRe = neig.T@J@meig
print(DiagRe[:2,:5])

normalization m & n
1.0000000000000004 5.147815070493501
eigenvalues: [-2.5+0.j]
gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.5001159174362994 1.0011440759202481 1.0
[[ 8.78677709e+00+4.51587058e-15j -7.21644966e-16-1.22631949e-15j
   2.18505769e-14+1.75410681e-14j -1.47104551e-15-4.07731544e-15j
   2.59514632e-15-2.60098714e-15j]
 [-4.36109482e-15-2.82162214e-15j  3.99037205e+00-3.27538799e-15j
  -4.47558657e-16+3.57927613e-15j -2.49800181e-15+2.87186089e-16j
  -2.96117297e-15+3.97881935e-15j]]


In [8]:
meig0=np.reshape(meig.copy()[:,0],(N,1))
neig0=np.reshape(neig.copy()[:,0],(N,1))
meig1=meig.copy()[:,1:]
neig1=neig.copy()[:,1:]
JRecon=meig0@neig0.T+meig1@neig1.T
print(JRecon[:3,:3])
print(J[:3,:3])

[[-0.01023754-1.51735811e-16j -0.01646705-4.37633935e-16j
   0.05172587-2.04971013e-16j]
 [-0.01646705+5.03733555e-16j -0.01868919+5.00618574e-16j
  -0.00027966+7.96863127e-16j]
 [ 0.05172587-1.13839935e-15j -0.00027966+3.03745882e-16j
  -0.01536811+1.63172549e-15j]]
[[-0.01023754 -0.01646705  0.05172587]
 [-0.01646705 -0.01868919 -0.00027966]
 [ 0.05172587 -0.00027966 -0.01536811]]


## heterogeneous $\eta$'s effects on changing of $\lambda$

In [13]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
## for all cases
''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# random gain and symmetricity
gaverage=1.0
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
nrank,ntrial,neta=1,10,len(symetaseries)

'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,-1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
signeta=np.ones(3)
# signeta[1]*=(-1)

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
Beigvseries=np.zeros((neta,ntrial,N,N))

for idxeta,eta in enumerate(symetaseries):
    for iktrial in range(ntrial):
        etaset=eta*np.ones(3)
        etaset[1]*=0.5
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        ''' HETEROGENEOUS ETA '''
        # EE#
        Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # EI/IE#
        Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # EE#
        Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0+(a-b)**2/(NE*2)

        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()

        Leig, Reig,Beigv = np.zeros((N,N)),np.zeros((N,N)),np.eye(N)
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        for i in range(N):
            Beigv[i,i]=normval[0,i]
        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:,:]=Beigv.copy()

gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
symetaseries: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.0000000000000004 5.147815070493501
eigenvalues: [-2.5+0.j]
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.00015585576561237264
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.0006271215845105198
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.001065216605915441
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 4.3596470834374887e-05
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 4.328429548399925e-05
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.00058662305067490

In [10]:
fig,ax=plt.subplots(figsize=(6,6))
ax.plot(symetaseries,np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1))

## different items
first_terms=np.zeros(ntrial)
second_terms = np.zeros(ntrial)
third_terms=np.zeros(ntrial,dtype=complex)
derivlambda=np.zeros(ntrial)
barn,barm=np.reshape(np.squeeze(neigvecAm[:,0]),(N,1)),np.reshape(np.squeeze(meigvecAm[:,0]),(N,1))
idxeta=1
for iktrial in range(ntrial):
    L1=np.squeeze(Leigvecseries[idxeta,iktrial,:,1:])
    R1=np.squeeze(Reigvecseries[idxeta,iktrial,:,1:])
    B1=np.squeeze(Beigvseries[idxeta,iktrial,1:,1:])
    l0=np.reshape(np.squeeze(Leigvecseries[idxeta,iktrial,:,0]),(N,1))
    r0=np.reshape(np.squeeze(Reigvecseries[idxeta,iktrial,:,0]),(N,1))
    first_terms[iktrial]=l0.T@(L1@B1@R1.T)@r0*(1/2.0)
    second_terms[iktrial]=l0.T@l0@r0.T@r0*Beigvseries[idxeta,iktrial,0,0]*1/2.0
    third_terms[iktrial]=np.squeeze((l0.T@barn)*(barm.T@r0))*1/2.0
    derivlambda[iktrial]=first_terms[iktrial]+second_terms[iktrial]-third_terms[iktrial]
print('first terms:',first_terms)
print('second terms:',second_terms)
print('third terms:',third_terms)
print('derivative of lambda:',derivlambda)

IndexError: too many indices for array

In [22]:
fig,ax=plt.subplots(figsize=(6,6))
ax.plot(symetaseries,np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1))

## different items
first_terms=np.zeros(ntrial)
second_terms = np.zeros(ntrial)
third_terms=np.zeros(ntrial,dtype=complex)
derivlambda=np.zeros(ntrial)
barn,barm=np.reshape(np.squeeze(neigvecAm[:,0]),(N,1)),np.reshape(np.squeeze(meigvecAm[:,0]),(N,1))
idxeta=1
for iktrial in range(ntrial):
    L1=np.squeeze(Leigvecseries[idxeta,iktrial,:,1:])
    R1=np.squeeze(Reigvecseries[idxeta,iktrial,:,1:])
    B1=np.squeeze(Beigvseries[idxeta,iktrial,1:,1:])
    l0=np.reshape(np.squeeze(Leigvecseries[idxeta,iktrial,:,0]),(N,1))
    r0=np.reshape(np.squeeze(Reigvecseries[idxeta,iktrial,:,0]),(N,1))
    first_terms[iktrial]=l0.T@(L1@B1@R1.T)@r0*(1/2.0)
    second_terms[iktrial]=l0.T@l0@r0.T@r0*Beigvseries[idxeta,iktrial,0,0]*1/2.0
    third_terms[iktrial]=np.squeeze((l0.T@barn)*(barm.T@r0))*1/2.0
    derivlambda[iktrial]=first_terms[iktrial]+second_terms[iktrial]-third_terms[iktrial]
print('first terms:',first_terms)
print('second terms:',second_terms)
print('third terms:',third_terms)
print('derivative of lambda:',derivlambda)

IndexError: too many indices for array

In [21]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((netadeux,ntrial,nrank),dtype=complex),np.zeros((netadeux,ntrial,NE*2,nrank),dtype=complex),np.zeros((netadeux,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((netadeux,ntrial,NE*2,nrank)),np.zeros((netadeux,ntrial,NE*2,nrank))
# properties
thetamaxseries=np.zeros((netadeux,ntrial,2,2))
pearsoncorr_rho=np.zeros((netadeux,ntrial,2,nrank))
covnmEI=np.zeros((netadeux,ntrial,2,nrank))
eta_est,eta_num=np.zeros((netadeux,ntrial)),np.zeros((netadeux,ntrial,2))

symetaseries = np.zeros(neta*2-1)
symetaseries[:neta]=symetaseriesUni[::-1].copy()
symetaseries[neta:]=symetaseriesUni[1:].copy()
signeta=np.zeros(3)
signeta[0::2]=1
signeta[1]=-1
'''

np.savez(now+'_perturbationPosLambdaHeterEta_data',Jparameters=Jparameters,symetaseries=symetaseries,etaset=etaset,signeta=signeta,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

## first method

In [20]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
## for all cases
''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# random gain and symmetricity
gaverage=1.0
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
nrank,ntrial,neta=1,10,len(symetaseries)

'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,-1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
signeta=np.ones(3)
# signeta[1]*=(-1)

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
Beigvseries=np.zeros((neta,ntrial,N))

for idxeta,eta in enumerate(symetaseries):
    for iktrial in range(ntrial):
        etaset=eta*np.ones(3)
        # etaset[1]*=0.50
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        ## EI IE##
        asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()

        # X0=Xinit.copy()*gaverage/np.sqrt(2)
        # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0

        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])

        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()

        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()

gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
symetaseries: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.0000000000000004 5.147815070493501
eigenvalues: [-2.5+0.j]
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.00014402510536913965
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.0006971301854027847
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.00013367292953621128
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 0.000600313744790883
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 2.4734528381629263e-05
compare the first and the last, mean, terms: (0.003124999999999968+0j) 6.2499999999999725
eta full: 0.0 ; num: 3.70784428752514e-

## The second rank in rank 2 network connectivity

In [15]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
## for all cases
''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# random gain and symmetricity
gaverage=1.0
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
nrank,ntrial,neta=1,10,len(symetaseries)

'''
## Three \bar{J} cases
'''
# JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
# a,b=0.0,0.0
JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
signeta=np.ones(3)
# signeta[1]*=(-1)

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
Beigvseries=np.zeros((neta,ntrial,N))

for idxeta,eta in enumerate(symetaseries):
    for iktrial in range(ntrial):
        etaset=eta*np.ones(3)
        etaset[1]*=0.50
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        ## EI IE##
        asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()

        # X0=Xinit.copy()*gaverage/np.sqrt(2)
        # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0

        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])

        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()

        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()


gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
symetaseries: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.0000000000000004 5.147815070493495
eigenvalues: [2.5+0.j]
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num: 0.0030864190427504863
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num: 0.0036843457478509638
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num: 0.004464201565804182
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num: 0.0036719543108185647
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num: 0.003413719953015234
compare the first and the last, mean, terms: (0.0031250000000001385+0j) 9.663546088957393e-34
eta full: 0.0 ; num:

In [14]:
fig,ax=plt.subplots(figsize=(6,6))
ax.plot(symetaseries,np.mean(np.squeeze(eigvalseries[:,:,0]),axis=1))

# ## different items
# first_terms=np.zeros(ntrial)
# second_terms = np.zeros(ntrial)
# third_terms=np.zeros(ntrial,dtype=complex)
# derivlambda=np.zeros(ntrial)
# barn,barm=np.reshape(np.squeeze(neigvecAm[:,0]),(N,1)),np.reshape(np.squeeze(meigvecAm[:,0]),(N,1))
# idxeta=0
# '''
# saving eigenvalues
# '''
# eigenvtrialavg = np.zeros((neta,N))
# for i in range(neta):
#     eigenvtrialavg[i,:]=np.mean(np.squeeze(i,:,:),axis=0)

# for iktrial in range(ntrial):
#     L1=np.squeeze(Leigvecseries[idxeta,iktrial,:,1:])
#     R1=np.squeeze(Reigvecseries[idxeta,iktrial,:,1:])
#     B1=np.diag(np.squeeze(eigenvtrialavg[ideta,:]))
#     # B1=np.diag(np.squeeze(Beigvseries[idxeta,iktrial,1:]))
#     l0=np.reshape(np.squeeze(Leigvecseries[idxeta,iktrial,:,0]),(N,1))
#     r0=np.reshape(np.squeeze(Reigvecseries[idxeta,iktrial,:,0]),(N,1))
#     first_terms[iktrial]=l0.T@(L1@B1@R1.T)@r0*(1/2.0)
#     second_terms[iktrial]=l0.T@l0@r0.T@r0*eigenvtrialavg[idxeta,0]*1/2.0
#     third_terms[iktrial]=np.squeeze((l0.T@barn)*(barm.T@r0))*1/2.0
#     derivlambda[iktrial]=first_terms[iktrial]+second_terms[iktrial]-third_terms[iktrial]
# print('first terms:',first_terms)
# print('second terms:',second_terms)
# print('third terms:',third_terms)
# print('derivative of lambda:',derivlambda)

In [13]:
## different items
l0_terms,r0_terms=np.zeros((NE*2,ntrial)),np.zeros((NE*2,ntrial))
barn,barm=np.reshape(np.squeeze(neigvecAm[:,0]),(N,1)),np.reshape(np.squeeze(meigvecAm[:,0]),(N,1))
lEInorm,rEInorm=np.zeros((2,ntrial)),np.zeros((2,ntrial))
lnEInorm,rmEInorm=np.zeros((2,ntrial)),np.zeros((2,ntrial))
idxeta=0
for iktrial in range(ntrial):
    L1=np.squeeze(Leigvecseries[idxeta,iktrial,:,1:])
    R1=np.squeeze(Reigvecseries[idxeta,iktrial,:,1:])
    B1=np.squeeze(Beigvseries[idxeta,iktrial,1:,1:])
    l0_terms[:,iktrial]=np.squeeze(Leigvecseries[idxeta,iktrial,:,0])
    r0_terms[:,iktrial]=np.squeeze(Reigvecseries[idxeta,iktrial,:,0])
    lEInorm[0,iktrial],lEInorm[1,iktrial]=np.sum(l0_terms[:NE,iktrial]**2),np.sum(l0_terms[NE:,iktrial]**2)
    rEInorm[0,iktrial],rEInorm[1,iktrial]=np.sum(r0_terms[:NE,iktrial]**2),np.sum(r0_terms[NE:,iktrial]**2)
    lnEInorm[0,iktrial],lnEInorm[1,iktrial]=np.sum(l0_terms[:NE,iktrial]*barn[:NE,0]),np.sum(l0_terms[NE:,iktrial]*barn[NE:,0])
    rmEInorm[0,iktrial],rmEInorm[1,iktrial]=np.sum(r0_terms[:NE,iktrial]*barm[:NE,0]),np.sum(r0_terms[NE:,iktrial]*barm[NE:,0])
    first_terms[iktrial]=l0.T@(L1@B1@R1.T)@r0*(1/2.0)
    second_terms[iktrial]=l0.T@l0@r0.T@r0*Beigvseries[idxeta,iktrial,0,0]*1/2.0
    third_terms[iktrial]=np.squeeze((l0.T@barn)*(barm.T@r0))*1/2.0
    derivlambda[iktrial]=first_terms[iktrial]+second_terms[iktrial]-third_terms[iktrial]
print('first terms:',first_terms)
print('second terms:',second_terms)
print('third terms:',third_terms)
print('derivative of lambda:',derivlambda)

first terms: [-0.86431901 -0.64225125 -0.31189891 -0.15360406 -0.08782373 -0.85910239
 -0.56985119 -0.24732879 -0.90997681 -0.67263092]
second terms: [7.72479973 8.07551799 7.1193844  7.24774195 6.48534277 7.45485257
 7.80118949 7.16973832 7.03658997 7.17669091]
third terms: [5.42319364+0.j 5.51337972+0.j 5.15393505+0.j 5.18299203+0.j
 4.89755081+0.j 5.30406914+0.j 5.42614989+0.j 5.17976186+0.j
 5.13343632+0.j 5.17944156+0.j]
derivative of lambda: [1.43728708 1.91988702 1.65355044 1.91114586 1.49996823 1.29168103
 1.80518841 1.74264767 0.99317684 1.32461843]


## perturbation theory -- analytical solution

In [35]:
# generate mean matrix
nrank=1
Nt=np.array([500,500])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
## for all cases
''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

# random gain and symmetricity
gaverage=1.0
# homogeneous 
grand=np.ones(4)*gaverage/np.sqrt(2)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
radius_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',radius_g)
'''
fixed g_bar but r_g changes (randomly insert)
eta changes and find out the relationships with eta, rotation and eigenvalue outliers
* symetaseries 
* gaverages =1.0
* grand fixed 
* ntrials
'''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
nrank,ntrial,neta=1,10,len(symetaseries)

'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
signeta=np.ones(3)
# signeta[1]*=(-1)

# original data
eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
Beigvseries=np.zeros((neta,ntrial,N))
statesReig,statesLeig=np.zeros((neta,ntrial,3,2)),np.zeros((neta,ntrial,3,2)) #E,I,Total/mean,std# 
gaverageseries=np.linspace(0.1,1.0,10)

for idxeta,eta in enumerate(gaverageseries):
    for iktrial in range(ntrial):
        etaset=np.zeros(3)#eta*np.ones(3)
        # etaset[1]*=0.50
        gaverage=eta
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        ## EI IE##
        asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()

        # X0=Xinit.copy()*gaverage/np.sqrt(2)
        # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0

        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])

        # ''' homogeneous '''
        X=Xinit.copy()
        X*=(gaverage/np.sqrt(2))
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()

        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])


gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
symetaseries: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.0000000000000004 5.147815070493516
eigenvalues: [-2.5+0.j]
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: -4.620124636693144e-06
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: -2.721884910259899e-07
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: -1.6512721715471423e-05
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: -1.2014548285065438e-05
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: 5.920381460954933e-05
compare the first and the last, mean, terms: (0.0062500000000000975+0j) 6.250000000000007
eta full: 0.0 ; num: 8.465275293690307e

In [28]:
fig,ax=plt.subplots(figsize=(6,6))
ax.plot(symetaseries,np.mean(np.squeeze(statesReig[:,:,0,1]),axis=1))

In [31]:
meanstd=np.mean(np.squeeze(statesReig[:,:,1,1]),axis=1)
print((meanstd[9]-meanstd[0])/(symetaseries[9]-symetaseries[0]))
print(1/np.sqrt(2*NE))

0.009017058714036689
0.022360679774997897


In [38]:
meanREstd=np.mean(np.squeeze(statesReig[:,:,0,0]),axis=1)
meanLEstd=np.mean(np.squeeze(statesLeig[:,:,0,0]),axis=1)
meanRIstd=np.mean(np.squeeze(statesReig[:,:,1,0]),axis=1)
meanLIstd=np.mean(np.squeeze(statesLeig[:,:,1,0]),axis=1)
print((meanLEstd[9]-meanLEstd[0])/(symetaseries[9]-symetaseries[0]))
print((meanLIstd[9]-meanLIstd[0])/(symetaseries[9]-symetaseries[0]))
print((meanLEstd[9]-meanLEstd[0])*(symetaseries[9]-symetaseries[0])/(symetaseries[9]-symetaseries[0])/(meanLIstd[9]-meanLIstd[0]))
print(1/np.sqrt(2*NE),JE/JI)

-0.027891802011550178
0.09710426232117156
-0.2872356098983407
0.03162277660168379 0.2857142857142857
